In [1]:
%run functions.ipynb

Ready: 2023-04-22 03:49:52


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()
spark

In [10]:
file = r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\10 городов\Мастер_таблица_2ГИС_10_городов_230419_1700_ext.xlsx"
city = 'ten_city'

In [11]:
plot_matrix(file, city, cnt_graphics=100, cnt_shars=12, step=1)

files/ten_city/ten_city_shars_230422_0811.docx


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [19:27<00:00, 11.67s/it]


In [127]:
df_tx_grouped_filtered = spark.read.parquet(f"files/ten_city/df_tx_grouped_filtered_ten_city")
df_tx_grouped_filtered.createOrReplaceTempView('df_tx_grouped_filtered')
rubrics = spark.sql('''select distinct rubricName from df_tx_grouped_filtered''').collect()
rubrics = [i.asDict()['rubricName'] for i in rubrics]
rubrics = '("' + '","'.join(rubrics) + '")'

In [13]:
df_schema = StructType([StructField('user', StringType(), True), StructField('proj', StringType(), True), StructField('rubricName', StringType(), True), StructField('rubricID', IntegerType(), True), StructField('time', StringType(), True), StructField('date', StringType(), True), StructField('org', StringType(), True), StructField('branch', StringType(), True), StructField('prod', StringType(), True), StructField('tx', StringType(), True), StructField('fl', StringType(), True), StructField('devmod', StringType(), True), StructField('lat', StringType(), True), StructField('lon', StringType(), True)])
spark.read.option("header","true").csv(files_list_chain[0], schema=df_schema).where('1=0').write.mode('overwrite').parquet(f'files/ten_city/requests_for_shars')

for file in tqdm(files_list_chain):
    city = file.split('\\')[2]
    final_df = spark.read.option("header","true").csv(file, schema=df_schema)
    final_df.createOrReplaceTempView('df')
    final_df = spark.sql(f'''
        select rubricName, year(time) as year_r, "{city}" as city, count(*) cnt_requests
        from df
        where rubricName in {rubrics}
        group by rubricName, year(time), "{city}"
    ''')
    
    final_df.write.mode('append').parquet(f'files/ten_city/requests_for_shars')

100%|███████████████████████████████████████████████████████████████████████████████| 40/40 [1:36:36<00:00, 144.90s/it]


In [140]:
def plot_matrix_cities(cnt_graphics=100, show=False):
    city='ten_city'
    shar_nums = list(range(1,16))*500
    time = str(datetime.datetime.now())[2:16].replace('-','').replace(' ','_').replace(':','')
    print(f'files/{city}/{city}_shars_{time}.docx')
    df_src = spark.read.parquet(f'files/ten_city/requests_for_shars').toPandas()
    df_new = df_src[['rubricName','city']].drop_duplicates().set_index(['rubricName','city'])
    df_new['Запросы 2020'] = df_src[df_src['year_r'] == 2020].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2021'] = df_src[df_src['year_r'] == 2021].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2022'] = df_src[df_src['year_r'] == 2022].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2023'] = df_src[df_src['year_r'] == 2023].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new = df_new.fillna(0).astype(int)
    df_new['Запросы, итого'] =  df_new['Запросы 2020'] + df_new['Запросы 2021'] + df_new['Запросы 2022'] + df_new['Запросы 2023']
    df_new = df_new.reset_index().set_index('city')

    df_tx_grouped_filtered = spark.read.parquet(f"files/ten_city/df_tx_grouped_filtered_ten_city")
    df_tx_grouped_filtered.createOrReplaceTempView('df_tx_grouped_filtered')
    rubrics = spark.sql('''select distinct rubricName from df_tx_grouped_filtered''').collect()
    rubrics = [i.asDict()['rubricName'] for i in rubrics]
    
    for rubric in tqdm(rubrics[:cnt_graphics]):
        shar_num = shar_nums.pop()
        df = df_new.loc[df_new['rubricName']==rubric]
        df['Запросы, итого'] = (df['Запросы, итого'] - min(df['Запросы, итого'])) / (max(df['Запросы, итого']) - min(df['Запросы, итого']))*300
        requests_total = df_new['Запросы 2023'].sum()
        df['Рост/Падение, %'] = df['Запросы 2023'] / df['Запросы 2022']
        df['Доля рынка, %'] = df['Запросы 2023'] / requests_total
        x = (df['Доля рынка, %'])
        y = (df['Рост/Падение, %'])
        sizes = (df['Запросы, итого']/1000)
        x_avg = (x.min()+x.max())/2
        y_avg = (y.min()+y.max())/2
        xi, yi = df['Доля рынка, %'], df['Рост/Падение, %']
        df.loc[(xi > x_avg) & (yi > y_avg), 'group'] = 'Звёзды:'
        df.loc[(xi > x_avg)  & (yi <= y_avg), 'group'] = 'Коровы:'
        df.loc[(xi <= x_avg)  & (yi > y_avg), 'group'] = 'Дети:'
        df.loc[(xi <= x_avg)  & (yi <= y_avg), 'group'] = 'Собаки:'
        df = df.sort_values(['Доля рынка, %', 'Рост/Падение, %'], ascending=[False, False])

        plt.rcParams['figure.figsize'] = 6.89*4, 4.25*4
        plt.rcParams['figure.dpi'] = 2**8
        plt.rcParams['font.size'] = '18'
        fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
        fig, ax = plt.subplots()
        ax.scatter(x, y, sizes, c='#C0504D', alpha=0.7)

        n = 0.02
        m = 0.02
        ax.set_ylim(ax.get_ylim()[0] * (1 - n), ax.get_ylim()[1] * (1 + n))
        ax.set_xlim(ax.get_xlim()[0] * (1 - m), ax.get_xlim()[1] * (1 + m))
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        plt.tick_params(bottom = False, left=False)
        plt.title(rubric)

        for xi, yi, rubric, size in zip(x, y, x.index, sizes):
            rs = rubric.split(' ')
            if len(rs) > 1:
                rsl = len(rs) // 2
                rubric = ' '.join(rs[:rsl] + ['\n'] + rs[rsl:])

            shar_path = [r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Шар_{:02}.png".format(shar_num)]
            shar_img = OffsetImage(plt.imread(shar_path[0], format="png"), zoom=size)
            ab = AnnotationBbox(shar_img, (xi, yi), frameon=False)
            ax.add_artist(ab)
            plt.annotate(rubric, (xi, yi), ha='center', va='center', font=fpath, color='#404040')

        ax.hlines((y.min()+y.max())/2, x.min(), x.max(), colors='gray', linewidth=1)
        ax.vlines((x.min()+x.max())/2, y.min(), y.max(), colors='gray', linewidth=1)
        
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354_gray.png")
        newax = fig.add_axes([0.1, 0.8, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354_gray.png")
        newax = fig.add_axes([0.9, 0.1, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        fig.savefig(f'pictures/{city}_shar_{shar_num}.png', bbox_inches='tight', transparent=True)
        
        txt = ''
        for cat in ['Звёзды:','Коровы:','Дети:','Собаки:']:
            txt +=  cat + ' ' + ', '.join([word[::-1].replace('/', 'и', 1)[::-1].replace('/', ',').replace(' ,', ',') for word in df[df['group'] == cat].index]) + "\n"
        txt = txt[::-1].replace('\n',':', 1)[::-1]
        add_image(f'files/{city}/{city}_shars_{time}.docx', f'pictures/{city}_shar_{shar_num}.png', word=txt)
        if show:
            plt.show()
        plt.close()

In [ ]:
plot_matrix_cities()

files/ten_city/ten_city_shars_230422_1720.docx


  7%|█████▋                                                                            | 7/100 [00:24<06:18,  4.07s/it]